In [ ]:
import pandas as pd
import numpy as np
import glob
import warnings

warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

# CARREGANDO A BASE PADRONIZADA
caminho = r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\BASES SEMANA PADRAO"
arquivos = glob.glob(caminho + "/*.parquet")

# Lista para armazenar DataFrames
dfList = []

# Lendo cada arquivo .parquet e adicionando à lista
for arquivo in arquivos:
    print(f"Lendo o arquivo: {arquivo}")
    df = pd.read_parquet(arquivo)
    dfList.append(df)

# Concatenando todos os DataFrames em um único DataFrame
df_telecobs_hugo = pd.concat(dfList, ignore_index=True)
df_telecobs_hugo


In [ ]:
#### Filtrando somente coluna telefone dos contratantes para os telefones que estiverem na base do contratante seja removido da base de higienizados
remover_telefones_contratante_do_higienizado = df_telecobs_hugo.loc[:,['TELEFONE']].drop_duplicates(['TELEFONE'])
## Criando coluna para identificar na base higienizados os telefones que são iguais aos contratante
remover_telefones_contratante_do_higienizado = remover_telefones_contratante_do_higienizado.assign(BASE_TEL_CONTRATANTE="CONTRATANTE")

In [ ]:
#################################################### CARERGANDO A BASE DE TELEFONES HIGIENIZADOS
caminho = r"R:\TI\TELEFONIA\BASES_HIGIENIZADAS_CLARO_E_NET\TELEFONES CLARO E NET"
arquivos = glob.glob(caminho + "/*.parquet")

# Lista para armazenar DataFrames
dfList = []

# Lendo cada arquivo .parquet e adicionando à lista
for arquivo in arquivos:
    print(f"Lendo o arquivo: {arquivo}")
    df = pd.read_parquet(arquivo)
    dfList.append(df)

# Concatenando todos os DataFrames em um único DataFrame
telefones_higienizados = pd.concat(dfList, ignore_index=True)
telefones_higienizados = telefones_higienizados.rename(columns={'TELEFONE':'TELEFONE_HIGIENIZADO'})

telefones_higienizados = pd.merge(telefones_higienizados, remover_telefones_contratante_do_higienizado, right_on=['TELEFONE'], left_on=['TELEFONE_HIGIENIZADO'], how="left").query("BASE_TEL_CONTRATANTE != 'CONTRATANTE'")\
.loc[:,['CPF','NOME','TELEFONE_HIGIENIZADO']]


In [ ]:
######################################################  BASE NORMALIZADA X TELEFONES HIGIENIZADOS  ######################################################

df_telecobs_hugo = pd.merge(df_telecobs_hugo, telefones_higienizados, right_on=['CPF'], left_on=['CPF'], how="left")\
.loc[:,['CPF','DEVEDOR','NOME_x','TELEFONE','EMAIL','ATRASO','SALDO_ABERTO','BASE','CONTRATO','TELEFONE_HIGIENIZADO']]

### REMOVENDO DUPLICADOS DEPOIS DO CRUZAMENTO, PARA EVITAR NOS PROXIMOS CRUZAMENTOS TER VARIOS CPF'S IGUAIS DESNCESSARIAMENTE.
df_telecobs_hugo = df_telecobs_hugo.drop_duplicates(['CPF','DEVEDOR', 'NOME_x', 'TELEFONE', 'EMAIL','ATRASO', 'BASE', 'TELEFONE_HIGIENIZADO'])

In [ ]:
#################################################### CARERGANDO A BASE DE TELEFONES HIGIENIZADOS
caminho = r"R:\TI\TELEFONIA\BASES_HIGIENIZADAS_CLARO_E_NET\EMAILS CLARO E NET"
arquivos = glob.glob(caminho + "/*.parquet")

# Lista para armazenar DataFrames
dfList = []

# Lendo cada arquivo .parquet e adicionando à lista
for arquivo in arquivos:
    print(f"Lendo o arquivo: {arquivo}")
    df = pd.read_parquet(arquivo)
    dfList.append(df)

# Concatenando todos os DataFrames em um único DataFrame
emails_higienizados = pd.concat(dfList, ignore_index=True)
emails_higienizados = emails_higienizados.rename(columns={'EMAIL':'EMAIL_HIGIENIZADO'})


In [ ]:
######################################################  BASE NORMALIZADA X EMAILS HIGIENIZADOS  ######################################################
df_telecobs_hugo = pd.merge(df_telecobs_hugo, emails_higienizados, right_on=['CPF'], left_on=['CPF'], how="left")\
.loc[:,['CPF','DEVEDOR','NOME_x','TELEFONE','EMAIL','ATRASO','SALDO_ABERTO','BASE','CONTRATO','TELEFONE_HIGIENIZADO','EMAIL_HIGIENIZADO']]

### REMOVENDO DUPLICADOS DEPOIS DO CRUZAMENTO, PARA EVITAR NOS PROXIMOS CRUZAMENTOS TER VARIOS CPF'S IGUAIS DESNCESSARIAMENTE.
df_telecobs_hugo = df_telecobs_hugo.drop_duplicates(['CPF','DEVEDOR', 'NOME_x', 'TELEFONE', 'EMAIL','ATRASO', 'BASE', 'TELEFONE_HIGIENIZADO', 'EMAIL_HIGIENIZADO'])

In [ ]:
### REMOVENDO DUPLICADOS DEPOIS DO CRUZAMENTO, PARA EVITAR NOS PROXIMOS CRUZAMENTOS TER VARIOS CPF'S IGUAIS DESNCESSARIAMENTE.
df_telecobs_hugo = df_telecobs_hugo.drop_duplicates(['CPF','DEVEDOR', 'NOME_x', 'TELEFONE', 'EMAIL','ATRASO', 'BASE','CONTRATO', 'TELEFONE_HIGIENIZADO','EMAIL_HIGIENIZADO'])
df_telecobs_hugo

In [ ]:
### SALVANDO OS TELEFONES HIGIENIZADOS EM PARQUET
df_telecobs_hugo.to_parquet(r'R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\BASES SEMANA PADRAO\BASE_OPERACIONAL/BASE_OPERACIONAL_TELECOB_HUGO.parquet', engine='pyarrow', compression='gzip')